# Görüntüleri okuma

In [ ]:

import cv2
import os 
import numpy as np

x = [] # işlenmiş görüntüler burada
y = [] # sınıf etiketleri

base_folder = r"D:\ilk gorev veri seti"

for root, dirs, files in os.walk(base_folder): # tüm alt klasörleri dolasir
    for file in files:
         if not file.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")):
            continue
         
         path = os.path.join(root, file) # gerçek dosya yolunu alıyor 

         resim = cv2.imread(path, cv2.IMREAD_GRAYSCALE) # gri tonlama işlemi 
         if resim is None:
             print("Okunamadi:",path)
             continue
         
         resim = cv2.GaussianBlur(resim,(5,5),0) # gürültüyü azaltır. model stabil özellikler sağlar.
         resim = cv2.resize(resim, (128,128)) # aynı boyut

         kontrast = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)) # kontrastı arttırır. Lezyon bölgeleri daha belirgin. 
         resim = kontrast.apply(resim)

         resim = resim.astype("float32") / 255.0 # piksel değerlerinin 0-1 aralıkta olmasını sağlayarak eğitimde karaalılık sağlar. 

         resim_flat = resim.flatten() # tek vektör, 


         x.append(resim) # yukarıda belirttiğimiz gibi x e ekledik 
         label = os.path.basename(os.path.dirname(path)) # etiketler otomatik çıkarılıyor. 

         y.append(label)

print("Toplam resim sayısı:", len(x))
print("Toplam etiket sayısı:", len(y))



       

Toplam resim sayısı: 6704
Toplam etiket sayısı: 6704


In [11]:
import sys
print("Bu Python kullanılıyor:", sys.executable)

# SKLEARN'İ TAM BU ORTAMA KUR
!"{sys.executable}" -m pip install scikit-learn


Bu Python kullanılıyor: c:\Users\User\AppData\Local\Programs\Python\Python313\python.exe



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Model oluşturup veriyi tahmin etme

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

x = np.array(x) # ml algoritmaları numpy array ile çalışır. Python listeleri uygun formatta değil. 
y = np.array(y)

x = x.reshape(len(x),-1) # görüntü boyutlarını tek boyutlu vektöre çevirmeliyiz okuyabilmesi için.

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=200) # 200 karar ağacı kullanır. (kısaca bu her ağaç eğitiliyor, tahmin yapıyor ve oylama sonucu çoğunluk ne diyorsa model o kararı veriyor)
model.fit(x_train, y_train)

tahmin = model.predict(x_test)
print("Basari: ", accuracy_score(y_test, tahmin))




Basari:  0.8538404175988069
